In [6]:
%load_ext autoreload
%autoreload 2

In [117]:
import numpy as np
import random 
import matplotlib.pyplot as plt
import argparse
import secrets
import json
import sys
import math 

In [108]:
from patient.simulator import run_multi_seed
from patient.baseline_policies import random_policy, greedy_policy
from patient.online_policies import p_approximation, p_approximation_balance, p_approximation_with_additions, p_approximation_with_additions_extra_provider, p_approximation_with_additions_no_match, p_approximation_with_additions_loose_constraints
from patient.offline_policies import offline_solution, offline_solution_balance, offline_learning_solution, offline_solution_more_patients, one_shot_policy, optimal_policy_epoch, offline_solution_loose_constraints, offline_solution_2_more_patients
from patient.utils import get_save_path, delete_duplicate_results, restrict_resources
from patient.learning import guess_coefficients

In [9]:
is_jupyter = 'ipykernel' in sys.modules

In [225]:
if is_jupyter: 
    seed        = 46
    num_patients = 2
    num_providers = 10
    provider_capacity = 1
    top_choice_prob = 0.5
    choice_model = "uniform_choice"
    utility_function = "normal"
    out_folder = "policy_comparison"
    exit_option = 0.5
    true_top_choice_prob = 0.5
    num_repetitions = 1
    num_trials = 100
    context_dim = 5
    max_menu_size = 5
else:
    parser = argparse.ArgumentParser()
    parser.add_argument('--seed', help='Random Seed', type=int, default=42)
    parser.add_argument('--n_patients',         '-N', help='Number of patients', type=int, default=100)
    parser.add_argument('--n_providers',        help='Number of providers', type=int, default=100)
    parser.add_argument('--n_trials',          help='Number of trials ', type=int, default=2)
    parser.add_argument('--top_choice_prob',          help='Probability of picking top choice', type=float, default=0.75)
    parser.add_argument('--true_top_choice_prob',          help='Probability of picking top choice', type=float, default=0.75)
    parser.add_argument('--context_dim',          help='Context dim for patients and providers', type=int, default=5)
    parser.add_argument('--max_menu_size',          help='Context dim for patients and providers', type=int, default=5)
    parser.add_argument('--num_repetitions',          help='Context dim for patients and providers', type=int, default=10)
    parser.add_argument('--provider_capacity', help='Provider Capacity', type=int, default=5)
    parser.add_argument('--choice_model', help='Which choice model for patients', type=str, default='uniform_choice')
    parser.add_argument('--exit_option', help='What is the value of the exit option', type=float, default=0.5)
    parser.add_argument('--out_folder', help='Which folder to write results to', type=str, default='policy_comparison')
    parser.add_argument('--utility_function', help='Which folder to write results to', type=str, default='uniform')

    args = parser.parse_args()

    seed = args.seed
    num_patients = args.n_patients
    num_providers = args.n_providers 
    provider_capacity = args.provider_capacity
    top_choice_prob = args.top_choice_prob
    choice_model = args.choice_model
    exit_option = args.exit_option
    out_folder = args.out_folder
    num_trials = args.n_trials 
    context_dim = args.context_dim 
    num_repetitions = args.num_repetitions
    true_top_choice_prob = args.true_top_choice_prob
    max_menu_size = args.max_menu_size
    utility_function = args.utility_function

save_name = secrets.token_hex(4)  

In [226]:
results = {}
results['parameters'] = {'seed'      : seed,
        'num_patients'    : num_patients,
        'num_providers': num_providers, 
        'provider_capacity'    : provider_capacity,
        'top_choice_prob': top_choice_prob, 
        'choice_model': choice_model,
        'exit_option': exit_option,
        'num_trials': num_trials,
        'context_dim': context_dim, 
        'true_top_choice_prob': true_top_choice_prob, 
        'num_repetitions': num_repetitions, 
        'max_menu_size': max_menu_size, 
        'utility_function': utility_function} 

## Baselines

In [235]:
seed_list = [43]
restrict_resources()

In [236]:
policy = random_policy
name = "random"
print("{} policy".format(name))

rewards, simulator = run_multi_seed(seed_list,policy,results['parameters'])

results['{}_matches'.format(name)] = rewards['matches']
results['{}_utilities'.format(name)] = rewards['patient_utilities']
results['{}_workloads'.format(name)] = rewards['provider_workloads']

np.sum(rewards['matches'])/(num_patients*num_trials*len(seed_list)),np.sum(rewards['patient_utilities'])/(num_patients*num_trials*len(seed_list))

random policy
Took 0.009081602096557617 time


/usr0/home/naveenr/projects/patient_provider/patient/simulator.py:245: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  


(0.54, 0.38792870156674786)

In [237]:
if 2**(num_patients*num_providers)*2**(num_patients)*math.factorial(num_patients) < 100000:
    policy = one_shot_policy
    per_epoch_function = optimal_policy_epoch
    name = "optimal"
    print("{} policy".format(name))

    rewards, simulator = run_multi_seed(seed_list,policy,results['parameters'],per_epoch_function)

    results['{}_matches'.format(name)] = rewards['matches']
    results['{}_utilities'.format(name)] = rewards['patient_utilities']
    results['{}_workloads'.format(name)] = rewards['provider_workloads']

    print(np.sum(rewards['matches'])/(num_patients*num_trials*len(seed_list)),np.sum(rewards['patient_utilities'])/(num_patients*num_trials*len(seed_list)))

In [238]:
policy = greedy_policy
name = "greedy"
print("{} policy".format(name))

rewards, simulator = run_multi_seed(seed_list,policy,results['parameters'])

results['{}_matches'.format(name)] = rewards['matches']
results['{}_utilities'.format(name)] = rewards['patient_utilities']
results['{}_workloads'.format(name)] = rewards['provider_workloads']

np.sum(rewards['matches'])/(num_patients*num_trials*len(seed_list)),np.sum(rewards['patient_utilities'])/(num_patients*num_trials*len(seed_list))

greedy policy
Took 0.009204387664794922 time


(0.54, 0.379609878983271)

## Offline

In [243]:
policy = one_shot_policy
per_epoch_function = offline_solution
name = "offline_solution"
print("{} policy".format(name))

rewards, simulator = run_multi_seed(seed_list,policy,results['parameters'],per_epoch_function)

results['{}_matches'.format(name)] = rewards['matches']
results['{}_utilities'.format(name)] = rewards['patient_utilities']
results['{}_workloads'.format(name)] = rewards['provider_workloads']

np.sum(rewards['matches'])/(num_patients*num_trials*len(seed_list)),np.sum(rewards['patient_utilities'])/(num_patients*num_trials*len(seed_list))

offline_solution policy
Took 0.005257129669189453 time


/usr0/home/naveenr/projects/patient_provider/patient/simulator.py:244: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  provider_workloads = [sum([len(j) for j in i])/len(i) for i in np.array(utilities_by_provider).T]


(0.54, 0.45065903407790076)

In [244]:
policy = one_shot_policy
per_epoch_function = offline_solution_loose_constraints
name = "offline_solution_loose_constraints"
print("{} policy".format(name))

rewards, simulator = run_multi_seed(seed_list,policy,results['parameters'],per_epoch_function)

results['{}_matches'.format(name)] = rewards['matches']
results['{}_utilities'.format(name)] = rewards['patient_utilities']
results['{}_workloads'.format(name)] = rewards['provider_workloads']

np.sum(rewards['matches'])/(num_patients*num_trials*len(seed_list)),np.sum(rewards['patient_utilities'])/(num_patients*num_trials*len(seed_list))

offline_solution_loose_constraints policy
Took 0.005906343460083008 time


(0.54, 0.45065903407790076)

In [256]:
policy = one_shot_policy
per_epoch_function = p_approximation_with_additions
name = "offline_solution_swaps"
print("{} policy".format(name))

rewards, simulator = run_multi_seed(seed_list,policy,results['parameters'],per_epoch_function)

results['{}_matches'.format(name)] = rewards['matches']
results['{}_utilities'.format(name)] = rewards['patient_utilities']
results['{}_workloads'.format(name)] = rewards['provider_workloads']

np.sum(rewards['matches'])/(num_patients*num_trials*len(seed_list)),np.sum(rewards['patient_utilities'])/(num_patients*num_trials*len(seed_list))

offline_solution_swaps policy
Weights [[0.38433646 0.67528645 0.71323352 0.61914661 0.53376962 0.20916116
  0.19921353 0.80265418 0.2947814  0.65686562]
 [0.39879103 0.67020226 0.71443222 0.61248071 0.53235454 0.21192458
  0.20892026 0.79408698 0.28659404 0.63980723]]
Matchings [[0. 0. 1. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 1. 0. 0.]]
Took 0.0053522586822509766 time


/usr0/home/naveenr/projects/patient_provider/patient/simulator.py:244: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  provider_workloads = [sum([len(j) for j in i])/len(i) for i in np.array(utilities_by_provider).T]


(0.54, 0.41877691552771656)

In [246]:
policy = one_shot_policy
per_epoch_function = p_approximation_with_additions_loose_constraints
name = "offline_solution_swaps_loose"
print("{} policy".format(name))

rewards, simulator = run_multi_seed(seed_list,policy,results['parameters'],per_epoch_function)

results['{}_matches'.format(name)] = rewards['matches']
results['{}_utilities'.format(name)] = rewards['patient_utilities']
results['{}_workloads'.format(name)] = rewards['provider_workloads']

np.sum(rewards['matches'])/(num_patients*num_trials*len(seed_list)),np.sum(rewards['patient_utilities'])/(num_patients*num_trials*len(seed_list))

offline_solution_swaps policy
Took 0.005379438400268555 time


(0.54, 0.45065903407790076)

In [247]:
policy = one_shot_policy
per_epoch_function = p_approximation_with_additions_no_match
name = "offline_solution_swaps_no_match"
print("{} policy".format(name))

rewards, simulator = run_multi_seed(seed_list,policy,results['parameters'],per_epoch_function)

results['{}_matches'.format(name)] = rewards['matches']
results['{}_utilities'.format(name)] = rewards['patient_utilities']
results['{}_workloads'.format(name)] = rewards['provider_workloads']

np.sum(rewards['matches'])/(num_patients*num_trials*len(seed_list)),np.sum(rewards['patient_utilities'])/(num_patients*num_trials*len(seed_list))

offline_solution_swaps_no_match policy
Took 0.005680561065673828 time


(0.54, 0.45065903407790076)

In [259]:
policy = one_shot_policy 
per_epoch_function = offline_solution_more_patients
name = "more_patients_than_providers"
print("{} policy".format(name))

rewards, simulator = run_multi_seed(seed_list,policy,results['parameters'],per_epoch_function)

results['{}_matches'.format(name)] = rewards['matches']
results['{}_utilities'.format(name)] = rewards['patient_utilities']
results['{}_workloads'.format(name)] = rewards['provider_workloads']

np.sum(rewards['matches'])/(num_patients*num_trials*len(seed_list)),np.sum(rewards['patient_utilities'])/(num_patients*num_trials*len(seed_list))

more_patients_than_providers policy
Took 0.006766080856323242 time


/usr0/home/naveenr/projects/patient_provider/patient/offline_policies.py:194: RuntimeWarning: invalid value encountered in double_scalars
  benefit_old = (total[provider])/(matches_by_provider[provider]) * (1-(1-p)**(matches_by_provider[provider]))


(0.54, 0.41877691552771656)

In [263]:
policy = one_shot_policy 
per_epoch_function = offline_solution_2_more_patients
name = "more_patients_than_providers_2"
print("{} policy".format(name))

rewards, simulator = run_multi_seed(seed_list,policy,results['parameters'],per_epoch_function)

results['{}_matches'.format(name)] = rewards['matches']
results['{}_utilities'.format(name)] = rewards['patient_utilities']
results['{}_workloads'.format(name)] = rewards['provider_workloads']

np.sum(rewards['matches'])/(num_patients*num_trials*len(seed_list)),np.sum(rewards['patient_utilities'])/(num_patients*num_trials*len(seed_list))

more_patients_than_providers_2 policy
[10. 10.]
Took 0.00624537467956543 time


(0.54, 0.379609878983271)

## Save Data

In [91]:
save_path = get_save_path(out_folder,save_name)

In [92]:
delete_duplicate_results(out_folder,"",results)

In [93]:
json.dump(results,open('../../results/'+save_path,'w'))